In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset 
import numpy as np 

In [2]:
def mnistIID(dataset, num_users):
    images = int(len(dataset)/num_users)
    users_dict, indeces = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        np.random.seed(i)
        users_dict[i] = set(np.random.choice(indeces, images, replace=False))
        indeces = list(set(indeces) - users_dict[i])
    return users_dict

In [3]:
def mnistNonIID(dataset, num_users, test=False):
    classes, images = 200, 300
    if test:
        classes, images = 20, 500
    classes_indx = [i for i in range(classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    indeces = np.arange(classes*images)
    unsorted_labels = dataset.train_labels.numpy()

    indeces_unsortedlabels = np.vstack((indeces, unsorted_labels))
    indeces_labels = indeces_unsortedlabels[:, indeces_unsortedlabels[1, :].argsort()]
    indeces = indeces_labels[0, :]

In [4]:
class FedDataset(Dataset):
    def __init__(self, dataset, indx):
        self.dataset = dataset
        self.indx = [int(i) for i in indx]
    
    def __len__(self):
        return len(self.indx)
    
    def __getitem__(self,item):
        images, label = self.dataset[self.indx[item]]
        return torch.tensor(images), torch.tensor(label)
        

In [5]:
def getActualImgs(dataset, indeces, batch_size):
    return DataLoader(FedDataset(dataset, indeces), batch_size = batch_size, shuffle = True)

In [ ]:
#Use these functions directly to some other file for implementation